In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import DBSCAN

In [5]:
#s = Data Path p = Number of Cluster col = Label Column Name
def detected_indices_temp(data,p,col,model):
    class0 = data.loc[data[col]!=1,:]
    class1 = data.loc[data[col]!=0,:]
    def noise(data):
        sample = data.sample(frac = 0.4)
        for i in list(sample.index):
            data[col][i] = int(not data[col][i])
        return list(sample.index)    
    noise0 = noise(class0)
    noise1 = noise(class1)
    noisy_indices = noise0+noise1
    mislabeled = pd.concat([class0,class1])
    indices = list(mislabeled.index)

    # User Optimal Number of Cluster
    km = KMeans(n_clusters = p)
    km.fit(mislabeled.loc[:,mislabeled.columns!=col])
    label = km.labels_
    mislabeled['cluster'] = label

    #OneHotEncoding
    mislabeled = pd.get_dummies(mislabeled,columns=['cluster'])
    #RandomForestClassifier
    x_train,x_test,y_train,y_test = train_test_split(mislabeled.loc[:,mislabeled.columns!=col],mislabeled[col],test_size = 0.38)
    rf = model()
    rf.fit(x_train,y_train)
    predict = rf.predict(mislabeled.loc[:,mislabeled.columns!=col])
    mislabeled['predicted'] = predict
    detected = list()
    cnt = 0
    bad_change = 0
    for i in indices:
        if mislabeled[col][i] != mislabeled['predicted'][i]:
            detected.append(i)
    for i in detected:
        if i in noisy_indices:
            cnt+=1
        else:
            bad_change+=1
    return pd.DataFrame(data={'Total Noise':[len(noisy_indices)], 'Total Detected labels':[len(detected)], 'True Identified labels':[cnt], 'False identified labels':[bad_change], 'Total Noise Ratio':len(noisy_indices)/data.shape[0], 'True Identified Ratio':cnt/len(noisy_indices)})

    print(len(noisy_indices),len(detected),cnt,bad_change)        

In [6]:
#Eblow shows k=18 (15-20) when not normalized
#Elbow shows k=14 when normalized

df = pd.read_csv('heart-disease.data',header=None, names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','label'])
label = df['label']
df.replace(r'?',np.nan,inplace=True)
df.fillna(df.median(),inplace=True)
df_std = StandardScaler().fit_transform(df)
df = pd.DataFrame(df_std)
df.head()
df.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','label']
df['label'] = label
df['label'] = df['label'].map({1:1,2:1,3:1,4:1,0:0})
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0.948726,0.686202,-2.251775,0.757525,-0.264900,2.394438,1.016684,0.017197,-0.696631,1.087338,2.274579,-0.711131,0.660004,0
1,1.392002,0.686202,0.877985,1.611220,0.760415,-0.417635,1.016684,-1.821905,1.435481,0.397182,0.649113,2.504881,-0.890238,1
2,1.392002,0.686202,0.877985,-0.665300,-0.342283,-0.417635,1.016684,-0.902354,1.435481,1.346147,0.649113,1.432877,1.176752,1
3,-1.932564,0.686202,-0.165268,-0.096170,0.063974,-0.417635,-0.996749,1.637359,-0.696631,2.122573,2.274579,-0.711131,-0.890238,0
4,-1.489288,-1.457296,-1.208521,-0.096170,-0.825922,-0.417635,1.016684,0.980537,-0.696631,0.310912,-0.976352,-0.711131,-0.890238,0


In [7]:
#10 30
#17 61
#27 91
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = detected_indices_temp(df,14,'label',model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
result

,Total Noise,Total Detected labels,True Identified labels,False identified labels,Total Noise Ratio,True Identified Ratio,Model
0,122,116,71,45,0.40264,0.581967,SVC
1,122,68,39,29,0.40264,0.319672,Random Forest
2,122,147,55,92,0.40264,0.450820,Naive Bayes
3,122,123,63,60,0.40264,0.516393,Logistic Regression
4,122,58,23,35,0.40264,0.188525,Decision Tree


In [11]:
df.loc[:,df.columns!='label'].head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,0.948726,0.686202,-2.251775,0.757525,-0.264900,2.394438,1.016684,0.017197,-0.696631,1.087338,2.274579,-0.711131,0.660004
1,1.392002,0.686202,0.877985,1.611220,0.760415,-0.417635,1.016684,-1.821905,1.435481,0.397182,0.649113,2.504881,-0.890238
2,1.392002,0.686202,0.877985,-0.665300,-0.342283,-0.417635,1.016684,-0.902354,1.435481,1.346147,0.649113,1.432877,1.176752
3,-1.932564,0.686202,-0.165268,-0.096170,0.063974,-0.417635,-0.996749,1.637359,-0.696631,2.122573,2.274579,-0.711131,-0.890238
4,-1.489288,-1.457296,-1.208521,-0.096170,-0.825922,-0.417635,1.016684,0.980537,-0.696631,0.310912,-0.976352,-0.711131,-0.890238


In [9]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,0.948726,0.686202,-2.251775,0.757525,-0.264900,2.394438,1.016684,0.017197,-0.696631,1.087338,2.274579,-0.711131,0.660004,0
1,1.392002,0.686202,0.877985,1.611220,0.760415,-0.417635,1.016684,-1.821905,1.435481,0.397182,0.649113,2.504881,-0.890238,1
2,1.392002,0.686202,0.877985,-0.665300,-0.342283,-0.417635,1.016684,-0.902354,1.435481,1.346147,0.649113,1.432877,1.176752,1
3,-1.932564,0.686202,-0.165268,-0.096170,0.063974,-0.417635,-0.996749,1.637359,-0.696631,2.122573,2.274579,-0.711131,-0.890238,0
4,-1.489288,-1.457296,-1.208521,-0.096170,-0.825922,-0.417635,1.016684,0.980537,-0.696631,0.310912,-0.976352,-0.711131,-0.890238,0


In [38]:
c = DBSCAN(eps=3,min_samples=3,metric='canberra').fit(df.loc[:,df.columns!='label'])

In [39]:
c.labels_

array([-1,  0,  0,  1,  1,  1,  3,  1,  0, -1, -1,  2, -1,  1,  1,  1, -1,
        1,  1,  1, -1, -1,  6, -1,  0,  1,  1, -1,  1,  0, -1, -1,  1, -1,
        1,  1,  0,  0,  0, -1, -1, -1,  1,  1,  1, -1,  1, -1, -1, -1,  1,
        1,  1,  1, -1,  0, -1,  1, -1, -1, -1, -1,  0, -1,  0,  0, -1, -1,
        0, -1,  1, -1,  0, -1,  1,  1,  0, -1,  1,  0,  0,  5,  1, -1,  1,
        1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1,  0,  3, -1,  1,  1,  1,
        1, -1, -1,  1,  1, -1,  0,  0, -1, -1,  6, -1, -1, -1,  1,  1, -1,
        0, -1,  3, -1, -1, -1,  1, -1,  0,  1,  1, -1, -1,  1,  1,  1,  2,
       -1, -1,  0, -1, -1,  4,  1, -1, -1,  1, -1,  1,  1,  1, -1,  5, -1,
        0, -1,  0, -1, -1, -1, -1,  1, -1,  1,  5,  1,  1,  1, -1,  1,  1,
       -1,  0,  7,  2,  0,  0, -1, -1, -1, -1, -1, -1, -1,  4,  1,  1, -1,
       -1, -1, -1,  1,  0, -1, -1, -1,  0, -1, -1,  1,  1,  1,  1, -1, -1,
        1, -1,  0,  0,  1, -1,  1, -1, -1, -1,  1,  6,  1, -1, -1,  1,  1,
        1,  1,  0, -1,  1